In [0]:
SELECT *
FROM
  system.billing.usage
WHERE
  usage_metadata.dlt_pipeline_id = :dlt_pipeline_id
  AND usage_start_time >= :usage_start_time
  AND usage_end_time < :usage_end_time
ORDER BY usage_date DESC

In [0]:
SELECT
  t1.usage_date,
   t1.workspace_id,
   SUM(t1.usage_quantity) AS `DBUs`,
   list_prices.pricing.default as list_price,
   SUM(t1.usage_quantity * list_prices.pricing.default) as list_cost
FROM system.billing.usage t1
INNER JOIN system.billing.list_prices on
   t1.cloud = list_prices.cloud and
   t1.sku_name = list_prices.sku_name and
   t1.usage_start_time >= list_prices.price_start_time and
   (t1.usage_end_time <= list_prices.price_end_time or list_prices.price_end_time is null)
WHERE
   t1.sku_name LIKE '%SERVERLESS%'
  --  AND billing_origin_product IN ("JOBS", "INTERACTIVE")
   AND t1.usage_date >= CURRENT_DATE() - INTERVAL 30 DAYS
   AND t1.usage_metadata.dlt_pipeline_id = :dlt_pipeline_id
GROUP BY ALL
ORDER BY t1.usage_date DESC

In [0]:
SELECT
   t1.*,
   list_prices.pricing.default as list_cost
FROM system.billing.usage t1
INNER JOIN system.billing.list_prices on
   t1.cloud = list_prices.cloud and
   t1.sku_name = list_prices.sku_name and
   t1.usage_start_time >= list_prices.price_start_time and
   (t1.usage_end_time <= list_prices.price_end_time or list_prices.price_end_time is null)
WHERE
   t1.sku_name LIKE '%SERVERLESS%'
  --  AND billing_origin_product IN ("JOBS", "INTERACTIVE")
   AND t1.usage_date >= CURRENT_DATE() - INTERVAL 30 DAYS
   AND t1.usage_metadata.dlt_pipeline_id = :dlt_pipeline_id
ORDER BY t1.usage_start_time DESC